In [ ]:
# !pip install koreanize-matplotlib

In [ ]:
import pandas as pd
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
# 데이터 컬럼 정보를 불러옵니다.
df_col = pd.read_csv("data/kadx-utf8-recipe-detail.csv")
df_col.shape

In [ ]:
df_col

In [ ]:
# 원본 데이터를 불러옵니다.
raw = pd.read_csv("data/kadx-utf8-recipe.csv")
raw.shape

In [ ]:
# 영문 컬럼명을 한글로 변환하기 위한 매핑 딕셔너리를 생성합니다.
col_dict = df_col[["nameEn","nameKo"]].set_index("nameEn")['nameKo'].to_dict()
col_dict

In [ ]:
# 원본 데이터의 컬럼명을 한글로 변환합니다.
df = raw.rename(columns=col_dict)

In [ ]:
# 변환된 데이터의 첫 몇 줄을 확인합니다.
df.head()

In [ ]:
# 기본 EDA
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
# 시각화: 조회수, 스크랩수에 대한 분포
sns.histplot(df['조회수'], kde=True)
plt.title('조회수 분포')
plt.show()

sns.histplot(df['스크랩수'], kde=True)
plt.title('스크랩수 분포')
plt.show()

In [ ]:
# 텍스트 분석: 요리소개와 재료내용을 TF-IDF로 변환
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['요리소개'])

# 군집 분석: KMeans 사용
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(tfidf_matrix)
df['Cluster'] = kmeans.labels_

# 군집 분석 결과 시각화 (PCA로 차원 축소)
pca = PCA(n_components=2)
pca_components = pca.fit_transform(tfidf_matrix.toarray())
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=df['Cluster'])
plt.title('KMeans Clustering of Recipes')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()

# 군집별 주요 키워드 나열
def get_top_keywords(tfidf_matrix, kmeans, vectorizer, n_terms=10):
    df_top_keywords = pd.DataFrame()
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()
    
    for i in range(kmeans.n_clusters):
        top_keywords = [terms[ind] for ind in order_centroids[i, :n_terms]]
        df_top_keywords[f'Cluster {i}'] = top_keywords
    
    return df_top_keywords

# 군집별 주요 키워드 출력
df_keywords = get_top_keywords(tfidf_matrix, kmeans, vectorizer)
print("\n군집별 주요 키워드:\n", df_keywords)

# 군집별 레시피 개수
print("\n군집별 레시피 개수:\n", df['Cluster'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
import numpy as np

# 한글 폰트 설정 (맥OS 기준, 윈도우의 경우 적절한 폰트 경로로 변경 필요)
from matplotlib import font_manager, rc
font_path = "/System/Library/Fonts/Supplemental/AppleGothic.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

# 워드 클라우드 생성 함수
def create_wordcloud(keywords, frequencies, title):
    wordcloud = WordCloud(font_path=font_path,
                          background_color='white',
                          width=800,
                          height=400,
                          max_font_size=150,
                          min_font_size=10).generate_from_frequencies(dict(zip(keywords, frequencies)))
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=20)
    plt.tight_layout(pad=0)
    plt.show()

# get_top_keywords 함수를 사용하여 각 군집의 주요 키워드 추출
df_keywords = get_top_keywords(tfidf_matrix, kmeans, vectorizer, n_terms=50)  # 상위 50개 키워드 추출

# 각 군집별 워드 클라우드 생성
for i in range(kmeans.n_clusters):
    cluster_keywords = df_keywords[f'Cluster {i}']
    
    # TF-IDF 가중치 계산
    cluster_center = kmeans.cluster_centers_[i]
    keyword_weights = [cluster_center[vectorizer.vocabulary_[keyword]] for keyword in cluster_keywords]
    
    # 워드 클라우드 생성
    create_wordcloud(cluster_keywords, keyword_weights, f'Cluster {i} Word Cloud')

# 전체 데이터에 대한 워드 클라우드 생성 (비교용)
all_keywords = [word for cluster in df_keywords.columns for word in df_keywords[cluster]]
all_weights = [1] * len(all_keywords)  # 모든 키워드에 동일한 가중치 부여
create_wordcloud(all_keywords, all_weights, 'Overall Word Cloud')